In [1]:
import cv2 as cv
import mediapipe as mp
import time
import numpy as np

In [2]:
def resize(img, new_w = None, new_h = None, frac = 1, interpolation = None):
    if frac == 1:
        return img
    if (new_w == None) or (new_h == None):
        new_h, new_w = int(img.shape[0]*frac), int(img.shape[1]*frac)
    
    if interpolation == None:
        interpolation = cv.INTER_LINEAR
    return cv.resize(img, (new_w, new_h), interpolation= interpolation)

def find_hands(img, draw = False):
    frameRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    result = hands.process(frameRGB)
    if draw:
        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    return img, result

def get_landmarks(img, found_hand,hand_number = 0, relative_coordinates = True):
    landmarks = []
    if found_hand.multi_hand_landmarks:
        hand = found_hand.multi_hand_landmarks[hand_number]
        #img_w, img_h = img.shape[0], img.shape[1]
        for id, lm in enumerate(hand.landmark):
            landmarks.append([id,lm.x, lm.y])
    return np.array(landmarks)

def draw_landmarks(img, landmarks, landmark_ids = None, s = 1, t = None, c = (255,0,0)):
    img_x, img_y = img.shape[0], img.shape[1]
    if t == None:
        t = s*2
    mask = np.zeros((landmarks.shape[0]), dtype = np.bool)
    for lm in landmark_ids:
        mask = mask + (landmarks[:,0] == lm)
    landmarks = landmarks[mask]
    for lm in landmarks:
            img = cv.circle(img, (int(img_y*lm[1]), int(img_x*lm[2])), s, c, t, cv.FILLED)    
    return img

def landmark_distances(landmarks, landmark_ids):
    points = []
    d = 1
    assert(len(landmark_ids) == 2)
    if landmarks != None:
        for lm in landmarks:
            if lm[0] in landmark_ids:
                points.append(np.array([lm[1], lm[2]]))
        d = np.linalg.norm(points[0] - points[1])
    return d

def find_angle(p1,p2,c = np.array([0,0])):
    p1 = (p1-c)[1:]
    p2 = (p2-c)[1:]
    n1 = np.linalg.norm(p1)
    n2 = np.linalg.norm(p2)
    if n1*n2 == 0:
        return 0
    return (p1@p2)/(np.linalg.norm(p1)*np.linalg.norm(p2))

def straight_index_cont(landmarks, thresh = .5):
    p1 = landmarks[landmarks[:,0]==8].squeeze() # 8 = Index Tip Landmark
    p2 = landmarks[landmarks[:,0]==5].squeeze() # 5 = Index Knuckle Landmark
    c = landmarks[landmarks[:,0]==0].squeeze() # 0 = Wrist Landmark
    # Check angle with wrist
    a1 = find_angle(
        p1 = landmarks[landmarks[:,0]==8].squeeze(),
        p2 = landmarks[landmarks[:,0]==0].squeeze(),
        c = landmarks[landmarks[:,0]==5].squeeze())
    # Check distance with knucle and wrist ratio
    d1 = np.linalg.norm((p1-p2)[1:])
    d2 = np.linalg.norm((p2-c)[1:])
    ratio = np.abs((d1/d2)-1)
    ratio_conf = 1-ratio
    straight = (np.abs(a1)*ratio_conf) > thresh
    return np.abs(a1)*ratio_conf, straight

def color_gradient(col, gradient, step):
    new_col = np.array(np.array(col) + np.array(gradient)*step).astype(int).tolist()
    return tuple(new_col)

In [3]:
capture = cv.VideoCapture('5-hand-exercises-to-increase-range-of-motion-n.mp4')

landmarks = None
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
i = 0
tic = 0
isTrue, frame = capture.read()
img_x, img_y = frame.shape[0], frame.shape[1]
while isTrue:
    try:
        if (i % 2) == 0:
            frame_small = resize(frame, frac = .5)
            frame_small, result = find_hands(frame_small, draw = False)
            landmarks = get_landmarks(frame_small, result)

        if (landmarks.shape[0] > 0):
            straight_conf, straight = straight_index_cont(landmarks)
            col = color_gradient(col = (0,0,255), gradient = (0,255,-255), step = straight_conf*1.5)
            veredicto = 'no extendido {:.1f}'.format(straight_conf)
            if straight:
                veredicto = 'extendido {:.1f}'.format(straight_conf)
            cv.putText(
                img = frame,
                text = 'Indice {}'.format(veredicto),
                org = (10,200),
                fontFace = cv.FONT_HERSHEY_COMPLEX,
                fontScale = .8,
                color = col,
                thickness = 2)
            frame = draw_landmarks(frame, landmarks, s = 3, landmark_ids = [8,5,0], c = col)
        if i > 2000:
            break
        toc = time.time()
        fps = str(int(1/(toc-tic)))
        tic = toc
        cv.putText(img = frame,
                   text = fps,
                   org = (10,40),
                   fontFace = cv.FONT_HERSHEY_COMPLEX,
                   fontScale = 1,
                   color = (255,0,0),
                   thickness = 2)
        cv.imshow('Video', frame)
        cv.waitKey(1)
        isTrue, frame = capture.read()
        i +=1
    except Exception as e:
        print('Error! {}'.format(e))
        break
capture.release()
cv.destroyAllWindows()

C:\Users\vadel\AppData\Local\Temp\ipykernel_8044\326270785.py:33: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros((landmarks.shape[0]), dtype = np.bool)
